In [2]:
import dowhy
import econml
from dowhy import CausalModel
import dowhy.datasets
import pandas as pd
import numpy as np


data = pd.read_csv('https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv', header = None)

col = ['treatment', 'y_factual', 'y_cfactual', 'mu0', 'mu1']

for i in range (1,26):
    col.append("x"+str(i))

data.columns = col
data = data.astype({'treatment': 'bool'}, copy=False)
print(data.head())

   treatment  y_factual  y_cfactual       mu0       mu1        x1        x2  \
0       True   5.599916    4.318780  3.268256  6.854457 -0.528603 -0.343455   
1      False   6.875856    7.856495  6.636059  7.562718 -1.736945 -1.802002   
2      False   2.996273    6.633952  1.570536  6.121617 -0.807451 -0.202946   
3      False   1.366206    5.697239  1.244738  5.889125  0.390083  0.596582   
4      False   1.963538    6.202582  1.685048  6.191994 -1.045229 -0.602710   

         x3        x4        x5  ...  x16  x17  x18  x19  x20  x21  x22  x23  \
0  1.128554  0.161703 -0.316603  ...    1    1    1    1    0    0    0    0   
1  0.383828  2.244320 -0.629189  ...    1    1    1    1    0    0    0    0   
2 -0.360898 -0.879606  0.808706  ...    1    0    1    1    0    0    0    0   
3 -1.850350 -0.879606 -0.004017  ...    1    0    1    1    0    0    0    0   
4  0.011465  0.161703  0.683672  ...    1    1    1    1    0    0    0    0   

   x24  x25  
0    0    0  
1    0    0  
2 

In [3]:
xs = ""
for i in range(1,26):
    xs += ("x"+str(i)+"+")
model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=xs.split('+')
        )

In [4]:
#Identify the causal effect
identified_estimand = model.identify_effect()
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(E[y_factual|x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22
d[treatment]                                                                  

                                       
,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16])
                                       
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16,U) = P(y_factual|treatment,x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16)

### Estimand : 2
Estimand name: iv
No such variable found!

### Estimand : 3
Estimand name: frontdoor
No such variable found!



In [5]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression", test_significance = True)

print(estimate)

print("Causal Estimate is " + str(estimate.value))

data_1 = data[data["treatment"]==1]
data_0 = data[data["treatment"]==0]
print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(E[y_factual|x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22
d[treatment]                                                                  

                                       
,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16])
                                       
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16,U) = P(y_factual|treatment,x2,x21,x18,x11,x17,x14,x8,x7,x3,x15,x20,x25,x6,x5,x22,x19,x1,x13,x23,x24,x4,x10,x12,x9,x16)

## Realized estimand
b: y_factual~treatment+x2+x21+x18+x11+x17+x14+x8+x7+x3+x15+x20+x25+x6+x5+x22+x19+x1+x13+x23+x24+x4+x10+x12+x9+x16
Target units: ate

## Estimate
Mean value: 3.928671

Distance Matching Estimator - doesn't recognise package

In [6]:
causal_estimate_dmatch = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.distance_matching",
                                              target_units="att",
                                              method_params={'distance_metric':"minkowski", 'p':2})
print(causal_estimate_dmatch)
print("Causal Estimate is " + str(causal_estimate_dmatch.value))

ImportError: distance_matching_estimator is not an existing causal estimator.

Propensity score stratification

In [7]:
causal_estimate_strat = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_stratification",
                                              target_units="att")
print(causal_estimate_strat)
print("Causal Estimate is " + str(causal_estimate_strat.value))

ValueError: Method requires strata with number of data points per treatment > clipping_threshold (=10). No such strata exists. Consider decreasing 'num_strata' or 'clipping_threshold' parameters.

Propensity score matching

In [32]:
causal_estimate_match = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_matching",
                                              target_units="atc")
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,
d[treatment]                                                                  

                                                 
x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22,U) = P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22)

## Realized estimand
b: y_factual~treatment+x9+x4+x15+x5+x1+x19+x16+x10+x2+x23+x11+x25+x20+x13+x17+x14+x21+x24+x18+x6+x12+x7+x3+x8+x22
Target units: atc

##

Weighting

In [33]:
causal_estimate_ipw = model.estimate_effect(identified_estimand,
                                            method_name="backdoor.propensity_score_weighting",
                                            target_units = "ate",
                                            method_params={"weighting_scheme":"ips_weight"})
print(causal_estimate_ipw)
print("Causal Estimate is " + str(causal_estimate_ipw.value))

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,
d[treatment]                                                                  

                                                 
x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22,U) = P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22)

## Realized estimand
b: y_factual~treatment+x9+x4+x15+x5+x1+x19+x16+x10+x2+x23+x11+x25+x20+x13+x17+x14+x21+x24+x18+x6+x12+x7+x3+x8+x22
Target units: ate

##

Regression discontinuity - something wrong here

In [38]:
causal_estimate_regdist = model.estimate_effect(identified_estimand,
        method_name="iv.regression_discontinuity", 
        method_params={'rd_variable_name':'Z1',
                       'rd_threshold_value':0.5,
                       'rd_bandwidth': 0.15})
# print(causal_estimate_regdist)
print("Causal Estimate is " + str(causal_estimate_regdist.value))

Causal Estimate is None


In [6]:
refute_results = model.refute_estimate(identified_estimand, estimate, method_name = "random_common_cause")
print(refute_results)

Refute: Add a Random Common Cause
Estimated effect:3.9286717508727103
New effect:3.927429668353104



In [7]:
# 2) EconML double machine learning estimator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor

In [40]:
# 1) Linear ITE/CATE estimator (in-built in DoWhy)
df = data
#The estimated effect of changing treatment from 0 to 1.
linear_estimate = model.estimate_effect(identified_estimand, 
                                        method_name="backdoor.linear_regression",
                                       control_value=0,
                                       treatment_value=1)
print(linear_estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,
d[treatment]                                                                  

                                                 
x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22,U) = P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22)

## Realized estimand
b: y_factual~treatment+x9+x4+x15+x5+x1+x19+x16+x10+x2+x23+x11+x25+x20+x13+x17+x14+x21+x24+x18+x6+x12+x7+x3+x8+x22
Target units: ate

##

In [44]:
dml_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML",
                                     control_value = 0,
                                     treatment_value = 1,
                                 target_units = "ate",  # condition used for CATE
                                 confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor(),
                                                              "model_final":LassoCV(fit_intercept=False),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=False)},
                                               "fit_params":{}})
print(dml_estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,
d[treatment]                                                                  

                                                 
x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22,U) = P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22)

## Realized estimand
b: y_factual~treatment+x9+x4+x15+x5+x1+x19+x16+x10+x2+x23+x11+x25+x20+x13+x17+x14+x21+x24+x18+x6+x12+x7+x3+x8+x22 | 
Target units: ate


In [46]:
dml_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML",
                                     control_value = 0,
                                     treatment_value = 1,
                                 target_units = 1,  # condition used for CATE
                                 confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor(),
                                                              "model_final":LassoCV(fit_intercept=False),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True)},
                                               "fit_params":{}})
print(dml_estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                                        
────────────(Expectation(y_factual|x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,
d[treatment]                                                                  

                                                 
x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22))
                                                 
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→y_factual then P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22,U) = P(y_factual|treatment,x9,x4,x15,x5,x1,x19,x16,x10,x2,x23,x11,x25,x20,x13,x17,x14,x21,x24,x18,x6,x12,x7,x3,x8,x22)

## Realized estimand
b: y_factual~treatment+x9+x4+x15+x5+x1+x19+x16+x10+x2+x23+x11+x25+x20+x13+x17+x14+x21+x24+x18+x6+x12+x7+x3+x8+x22 | 
Target units: 

##

In [1]:
import keras

from econml.deepiv import DeepIVEstimator

dims_zx = len(model.get_instruments())+len(model.get_effect_modifiers())
dims_tx = len(model._treatment)+len(model.get_effect_modifiers())
treatment_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(dims_zx,)), # sum of dims of Z and X
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])
response_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(dims_tx,)), # sum of dims of T and X
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(1)])

deepiv_estimate = model.estimate_effect(identified_estimand,
                                        method_name="iv.econml.deepiv.DeepIV",
                                        target_units = lambda df: df["X0"]>-1,
                                        confidence_intervals=False,
                                method_params={"init_params":{'n_components': 10, # Number of gaussians in the mixture density networks
                                                              'm': lambda z, x: treatment_model(keras.layers.concatenate([z, x])), # Treatment model,
                                                              "h": lambda t, x: response_model(keras.layers.concatenate([t, x])), # Response model
                                                              'n_samples': 1, # Number of samples used to estimate the response
                                                              'first_stage_options': {'epochs':25},
                                                              'second_stage_options': {'epochs':25}
                                                             },
                                               "fit_params":{}})
print(deepiv_estimate)

ModuleNotFoundError: No module named 'tensorflow'

Not needed for our purposes

In [21]:
BETA = 5

data_binary = dowhy.datasets.linear_dataset(BETA, num_common_causes=4, num_samples=10000,
                                    num_instruments=2, num_effect_modifiers=2,
                                    treatment_is_binary=True, outcome_is_binary=True)

# convert boolean values to {0,1} numeric
data_binary['df'].v0 = data_binary['df'].v0.astype(int)
data_binary['df'].y = data_binary['df'].y.astype(int)
print(data_binary['df'])

model_binary = CausalModel(data=data_binary["df"],
                    treatment=data_binary["treatment_name"], outcome=data_binary["outcome_name"],
                    graph=data_binary["gml_graph"])
identified_estimand_binary = model_binary.identify_effect(proceed_when_unidentifiable=True)

            X0        X1   Z0        Z1        W0        W1        W2  \
0     0.797900 -1.524757  1.0  0.133343 -0.971123  0.570611  1.176735   
1    -1.293863  0.240758  0.0  0.176152  0.784583  1.001440  2.175154   
2     0.737523  0.499241  0.0  0.471321  1.701553  0.791216 -0.667507   
3     0.894083 -0.982632  1.0  0.074063  2.465837 -0.503028 -0.458893   
4    -0.605267 -1.631641  1.0  0.927050 -0.782338  0.945847 -0.276427   
...        ...       ...  ...       ...       ...       ...       ...   
9995 -1.009136 -0.048195  1.0  0.130304  0.400930 -0.116123  0.610205   
9996  0.407634 -1.070540  1.0  0.034995  0.972761 -0.011319 -0.712027   
9997  1.189302 -0.226554  1.0  0.262504  1.437425  0.899275  2.058467   
9998 -2.442810 -0.850211  0.0  0.694285  0.641811  1.354751  0.588460   
9999  1.473420  0.403831  1.0  0.674738  0.865393  0.849021  1.226882   

            W3  v0  y  
0     1.377399   1  1  
1     2.731591   1  1  
2     0.322772   1  1  
3    -0.943244   1  1  
4  

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
#todo needs binary y
drlearner_estimate = model_binary.estimate_effect(identified_estimand_binary,
                                method_name="backdoor.econml.drlearner.LinearDRLearner",
                                confidence_intervals=False,
                                method_params={"init_params":{
                                                    'model_propensity': LogisticRegressionCV(cv=3, solver='lbfgs', multi_class='auto')
                                                    },
                                               "fit_params":{}
                                              })
print(drlearner_estimate)
print("True causal estimate is", data_binary["ate"])